In [ ]:
! git clone https://github.com/huggingface/diffusers.git

In [1]:
import torch
import matplotlib.pyplot as plt
import random
import threading
from librosa.beat import beat_track
from pythonosc import dispatcher, osc_server
import numpy as np
from datasets import load_dataset
from diffusers import DiffusionPipeline, DDIMScheduler, AudioDiffusionPipeline
from diffusers_local import UNet2DModel as UNet2DModel_local
import soundfile as sf
from CreateDataset import AudioProcessor
from NetworkBending import NetworkBending
import time

osc_lock = threading.Lock()

NB = NetworkBending()

device = "cuda" if torch.cuda.is_available() else "cpu"
audio_diffusion = DiffusionPipeline.from_pretrained("teticio/audio-diffusion-ddim-256")
# apply the custom unet model so we can Networkbend the audio
audio_diffusion.unet = UNet2DModel_local.from_pretrained("teticio/audio-diffusion-ddim-256", subfolder="unet", network_bending=NB)

#set the scheduler to DDIM
audio_diffusion.scheduler = DDIMScheduler.from_pretrained("teticio/audio-diffusion-ddim-256", subfolder="scheduler")
audio_diffusion.to(device)

#ds = load_dataset('teticio/audio-diffusion-256')
#generator=torch.Generator(device="cpu").manual_seed(42)

alpha = 0
latent1 = 0
latent2 = 1
diffusion_steps = 10
make_loop = 0
generate = 1

c:\Users\danhearn\.conda\envs\diffusertrack\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\danhearn\.conda\envs\diffusertrack\Lib\site-packages\kornia\feature\lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
c:\Users\danhearn\.conda\envs\diffusertrack\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
unet\diffusion_pytorch_model.safetensors not found
Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]An error occurred while trying to fetch C:\Users\danhear

In [13]:
from melgan.modules import Generator_melgan
import yaml


MELGAN_MODEL_NAME = "best_netG.pt"

def read_yaml(filepath):
    with open(filepath, 'r') as file:
        return yaml.safe_load(file)

# Load a MelGAN model
def load_vocoder(device_name):
    feat_dim = 80
    mean_fp = f'data/drumbeats_1bar/mean.mel.npy'
    std_fp = f'data/drumbeats_1bar/std.mel.npy'
    v_mean = torch.from_numpy(np.load(mean_fp)).float().view(1, feat_dim, 1).to(device_name)
    v_std = torch.from_numpy(np.load(std_fp)).float().view(1, feat_dim, 1).to(device_name)

    n_mel_channels = 80
    ngf = 32
    n_residual_layers = 3

    vocoder = Generator_melgan(n_mel_channels, ngf, n_residual_layers).to(device_name)
    vocoder.eval()

    vocoder_param_fp = f'melgan/{MELGAN_MODEL_NAME}'
    vocoder.load_state_dict(torch.load(vocoder_param_fp, map_location=torch.device(device_name)), strict=False)

    return vocoder, v_mean, v_std

VOCODER, V_MEAN, V_STD = load_vocoder(device)

def vocode(sample, vocoder=VOCODER, v_mean=V_MEAN, v_std=V_STD):
    de_norm = sample.squeeze(0) * v_std + v_mean
    audio_output = vocoder(de_norm)
    return audio_output

C:\Users\danhearn\AppData\Local\Temp\ipykernel_35904\96833784.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vocoder.load_state_dict(torch.load(vocoder_param_fp, map_l

In [26]:
import torch
import numpy as np
from PIL import Image

# Load the PNG image as a normalized tensor
def load_image_as_tensor(image_path, device, target_dim=(80, 256)):
    # Load image and convert to grayscale if necessary
    image = Image.open(image_path).convert('L')  # Convert to grayscale

    # Resize image to the target dimensions (e.g., 80x256)
    image = image.resize(target_dim, Image.LANCZOS)
    
    # Convert image to numpy array, normalize it between -1 and 1
    image_np = np.array(image).astype(np.float32)
    image_np = (image_np / 255.0) * 2 - 1  # Normalize to match vocoder's expected input range

    # Convert numpy array to a torch tensor and add batch and channel dimensions
    image_tensor = torch.tensor(image_np).unsqueeze(0).unsqueeze(0).to(device)
    
    return image_tensor

# Load and process the image
image_path = 'generated_image.png'  # Path to your PNG image
image_tensor = load_image_as_tensor(image_path, device)

# Ensure the tensor has the correct dimensions
if image_tensor.shape[2] != V_MEAN.shape[2]:
    image_tensor = torch.nn.functional.interpolate(image_tensor, size=(V_MEAN.shape[2], image_tensor.shape[3]), mode='bilinear', align_corners=False)

# Run the vocoder to generate audio
audio_output = vocode(image_tensor).squeeze(0).cpu().detach().numpy()

# Ensure audio_output is 2D (samples, channels)
if audio_output.ndim == 1:
    audio_output = audio_output[:, np.newaxis]

# Ensure audio_output is float32
audio_output = audio_output.astype(np.float32)

sf.write("mel_gan.wav", audio_output, 22050)


LibsndfileError: Error opening 'mel_gan.wav': Format not recognised.

In [ ]:
# Load audio and process it into 5 second spectrograms
input_dir = r"C:\\Users\\dhearn\\Music"
processor = AudioProcessor(input_dir)
processor.process_directory()

In [ ]:
def loop_it(audio: np.ndarray,
        sample_rate: int,
        loops: int = 1) -> np.ndarray:
    """Loop audio

    Args:
        audio (np.ndarray): audio as numpy array
        sample_rate (int): sample rate of audio
        loops (int): number of times to loop

    Returns:
        (float, np.ndarray): sample rate and raw audio or None
    """
    _, beats = beat_track(y=audio, sr=sample_rate, units='samples')
    for beats_in_bar in [16, 12, 8, 4]:
        if len(beats) > beats_in_bar:
            return np.tile(audio[beats[0]:beats[beats_in_bar]], loops)
    return None

def interpolation(address, *args):
    
    """Interpolate between two latents for generation
    Args:
        address (str): OSC address
        *args: OSC arguments
    """

    global alpha, latent1, latent2
    
    if address == "/alpha":
        alpha = args[0]
    if address == "/latent1" and latent2 != args[0]:
        latent1 = int(args[0])
    if address == "/latent2" and latent1 != args[0]:
        latent2 = int(args[0])

def inference(address, *args):

    """Control the inference process
    Args:
        address (str): OSC address
        *args: OSC arguments
    """

    global diffusion_steps, make_loop, generate

    if address == "/steps":
        diffusion_steps = int(args[0])
    if address == "/loop":
        make_loop = int(args[0])
    if address == "/generate":
        generate = int(args[0])

In [ ]:
#load 20 images from the default dataset
ds = load_dataset('teticio/audio-diffusion-256')
images = []

for i in range(20):
    images.append(random.choice(ds['train'])['image'])

images[0].show()


In [ ]:
import os
from PIL import Image
import torch
import numpy as np

# Assuming `audio_diffusion` is your Diffusion model pipeline
encoded_images = []

# Directory containing mel spectrograms
spectrogram_dir = "data/mel_spectrograms"

# Loop through each file in the directory
for i, file_name in enumerate(os.listdir(spectrogram_dir)):
    spectrogram_path = os.path.join(spectrogram_dir, file_name)
    spectrogram_image = Image.open(spectrogram_path).convert("L")
    encoded_image = audio_diffusion.encode([spectrogram_image], steps=100)
    encoded_images.append(encoded_image)
    if len(encoded_images) >= 10:
        break


In [ ]:
for encoded_image in encoded_images:
    plt.imshow(encoded_image.cpu().squeeze(), cmap='gray')
    plt.axis('off')  # Hide axes
    plt.show()
    

In [ ]:
#osc listener
from pythonosc import dispatcher, osc_server

ip="127.0.0.1"
port=9999

d = dispatcher.Dispatcher()
# Map incoming OSC messages to the osc_receive method
d.map("/rotate_x_radian",  NB.osc_receive)
d.map("/rotate_y_radian", NB.osc_receive)
d.map("/rotate_z_radian", NB.osc_receive)
d.map("/rotate_x_scaling_factor", NB.osc_receive)
d.map("/rotate_y_scaling_factor", NB.osc_receive)
d.map("/rotate_z_scaling_factor", NB.osc_receive)
d.map("/scale_factor", NB.osc_receive)
d.map("/layer", NB.osc_receive)
d.map("/scale", NB.osc_receive)
d.map("/reflect", NB.osc_receive)
d.map("/erosion", NB.osc_receive)
d.map("/dilation", NB.osc_receive)
d.map("/gradient", NB.osc_receive)
d.map("/sobel", NB.osc_receive)
d.map("/add_rand_rows", NB.osc_receive)
d.map("/normalize", NB.osc_receive)
d.map("/rotate_x", NB.osc_receive)
d.map("/rotate_y", NB.osc_receive)
d.map("/rotate_z", NB.osc_receive)
d.map("/alpha", interpolation)
d.map("/latent1", interpolation)
d.map("/latent2", interpolation)
d.map("/steps", inference)
d.map("/loop", inference)
d.map("/generate", inference)

s = osc_server.ThreadingOSCUDPServer((ip, port), d)

osc_thread = threading.Thread(target=s.serve_forever)
osc_thread.daemon = True # This will allow the main program to exit even if the OSC server is still running
osc_thread.start()
print("OSC server started - listening on port 9999")

In [ ]:
# Main loop for image generation
try:
    while True:
        with osc_lock: 

            if generate == 1:
             
                output = audio_diffusion(steps=diffusion_steps,
                noise=AudioDiffusionPipeline.slerp(encoded_images[latent1], encoded_images[latent2], alpha), eta=0)

                output.images[0].save("generated_image.png")
                audio = output.audios[0, 0]

                if make_loop == 1:
                    loop = loop_it(audio, audio_diffusion.mel.get_sample_rate())
                    sf.write("output_audio.wav", loop, audio_diffusion.mel.get_sample_rate())
                elif make_loop == 0:
                    time.sleep(0.05)
                    sf.write("output_audio.wav", audio, audio_diffusion.mel.get_sample_rate())



except KeyboardInterrupt:
    s.shutdown()
    print("Terminating the loop gracefully.")